In [ ]:
%pip install pycaret



In [ ]:
import kagglehub
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pycaret.regression import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error,r2_score

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ignaciovinuales/battery-remaining-useful-life-rul")

In [ ]:
files_in_directory = os.listdir(path)
print(files_in_directory)

In [ ]:
HNEI_df = pd.read_csv(path+ '/Battery_RUL.csv')
HNEI_df.head()

In [ ]:
battery_id = 0

def add_id(Cycle_Index):
  global battery_id
  if Cycle_Index == 1:
    battery_id = battery_id +1
  return battery_id


HNEI_df['battery_id'] = HNEI_df['Cycle_Index'].apply(add_id)


In [ ]:
HNEI_df.drop(['Cycle_Index'], axis = 1, inplace = True)

In [ ]:
from sklearn.ensemble import IsolationForest

iso = IsolationForest(contamination=0.01, random_state=42)
outlier_labels = iso.fit_predict(HNEI_df.select_dtypes(include=['float', 'int']))
HNEI_df['outlier'] = outlier_labels
HNEI_df[HNEI_df['outlier'] == -1]

In [ ]:

outliers_df.describe()


In [ ]:
HNEI_df.describe()

In [ ]:
import seaborn as sns

sns.boxplot(x=HNEI_df['RUL'])
plt.title("RUL Boxplot")


In [ ]:
HNEI_df_filtered = HNEI_df[HNEI_df['outlier'] != -1]

In [ ]:
#I want to see if battery Id indeed leaks data
HNEI_df_test = HNEI_df_filtered[HNEI_df_filtered['battery_id'] == 1]
HNEI_train = HNEI_df_filtered[HNEI_df_filtered['battery_id'] != 1]

In [ ]:
model = setup(data = HNEI_train, target = 'RUL', session_id = 123)

In [ ]:
best_model = compare_models()

In [ ]:
et = create_model('et')

In [ ]:
predictions = predict_model(et, data = HNEI_df_test)

In [ ]:
predictions.head()

In [ ]:
#I feel like I could make a pipeline that takes data trains and tests

def pipeline(train_data, test_data):
  model = setup(data = train_data, target = 'RUL', session_id = 123)
  best_model = compare_models()
  predictions = predict_model(best_model, data = test_data)
  return best_model, predictions

In [ ]:
HNEI_df_test.drop(['battery_id'], axis = 1, inplace = True)
HNEI_train.drop(['battery_id'], axis = 1, inplace = True)

best_model, predictions = pipeline(HNEI_train,HNEI_df_test)

In [ ]:
# Visualize residuals
plot_model(best_model, plot='residuals')

# Prediction error (predicted vs actual)
plot_model(best_model, plot='error')

# Feature importance
plot_model(best_model, plot='feature')

# Cumulative residual distribution
plot_model(best_model, plot='cooks')



In [ ]:
HNEI_df_test.corr()['RUL'].sort_values(key = np.abs, ascending= False)

In [ ]:
HNEI_df_test.columns

In [ ]:
HNEI_train.drop(['Charging time (s)', 'Discharge Time (s)', 'Decrement 3.6-3.4V (s)'], axis=1, inplace = True)
HNEI_df_test.drop(['Charging time (s)', 'Discharge Time (s)', 'Decrement 3.6-3.4V (s)'], axis = 1, inplace = True)

pipeline(HNEI_train,HNEI_df_test)

In [ ]:
HNEI_df_test.head()

In [ ]:
error = y_pred - y_test

plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test, y=error, alpha=0.6)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Actual RUL')
plt.ylabel('Prediction Error (Predicted - Actual)')
plt.title('Prediction Error vs Actual RUL')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
HNEI_df_test['error'] = y_pred - y_test

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(x='battery_id', y='error', data=df)
plt.axhline(0, color='red', linestyle='--')
plt.title('Prediction Error by Battery ID')
plt.ylabel('Error (Predicted - Actual)')
plt.xlabel('Battery ID')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
HNEI_df_filtered

In [ ]:
#let's make the indices for the test sets

test_indices = []

for i in range (1,15):
   test_index = test_indices.append(HNEI_df_filtered[HNEI_df_filtered['battery_id'] == i].tail(200).index)

# Flatten the list of Index objects into a single list of indices
flat_test_indices = [idx for index_list in test_indices for idx in index_list]

# Select the rows using the flattened list of indices
eol_split_test = HNEI_df_filtered.loc[flat_test_indices]
eol_split_train = HNEI_df_filtered.drop(flat_test_indices)


In [ ]:
eol_split_test.drop(['battery_id', 'outlier'], axis = 1, inplace = True)
eol_split_train.drop(['battery_id', 'outlier'], axis = 1, inplace = True)

pipeline(eol_split_train, eol_split_test)

In [ ]:
predictions.tail(10)

In [ ]:
# Visualize residuals
plot_model(best_model, plot='residuals')

# Prediction error (predicted vs actual)
plot_model(best_model, plot='error')

# Feature importance
plot_model(best_model, plot='feature')

# Cumulative residual distribution
plot_model(best_model, plot='cooks')



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming y_test and y_pred are your actual and predicted RUL values
y_test = predictions['RUL']
y_pred = predictions['prediction_label']

plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test, y=y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', label='Ideal Prediction')
plt.xlabel('Actual RUL')
plt.ylabel('Predicted RUL')
plt.title('Predicted vs Actual RUL')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
error = y_pred - y_test

plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test, y=error, alpha=0.6)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Actual RUL')
plt.ylabel('Prediction Error (Predicted - Actual)')
plt.title('Prediction Error vs Actual RUL')
plt.grid(True)
plt.tight_layout()
plt.show()
